<a href="https://colab.research.google.com/github/hendrikyong/CVNL_Assignment_1/blob/main/CVNL_P02_GP01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
'''
1. import dataset

2. reorganise file for consistency (data preprocessing)
data provided was
Train set
Folder A
- A1.jpg
- A2.jpg
etc

Test set
- A_test.jpg
- B_test.jpg
etc

reorganising it into
Train set
Folder A
- A1.jpg
- A2.jpg
Folder B
- B1.jpg
- B2.jpg
etc

Test set
Folder A
- A_test.jpg
Folder B
- B_test.jpg
etc

3. define paths to the training and testing data directories
'''

'\n1. import dataset\n\n2. reorganise file for consistency (data preprocessing)\ndata provided was \nTrain set \nFolder A \n- A1.jpg\n- A2.jpg\netc\n\nTest set\n- A_test.jpg\n- B_test.jpg\netc\n\nreorganising it into \nTrain set \nFolder A \n- A1.jpg\n- A2.jpg\nFolder B\n- B1.jpg\n- B2.jpg\netc\n\nTest set\nFolder A\n- A_test.jpg\nFolder B\n- B_test.jpg\netc\n\n3. define paths to the training and testing data directories\n'

In [11]:
#importing dataset from kaggle
import kagglehub
import os

#download latest version
path = kagglehub.dataset_download("grassknoted/asl-alphabet")

#list files in the dataset folder
print("Path to dataset files:", path)
files = os.listdir(path)
print("Files in the dataset:", files)

Path to dataset files: /root/.cache/kagglehub/datasets/grassknoted/asl-alphabet/versions/1
Files in the dataset: ['asl_alphabet_train', 'asl_alphabet_test']


In [12]:
#define paths to the training and testing data directories
train_dir = '/root/.cache/kagglehub/datasets/grassknoted/asl-alphabet/versions/1/asl_alphabet_train/asl_alphabet_train'
test_dir = '/root/.cache/kagglehub/datasets/grassknoted/asl-alphabet/versions/1/asl_alphabet_test/asl_alphabet_test'

In [13]:
#reorganise files
#what is needed to be done is that from the test_dir for each image get first letter since all images are
#labelled as letter_test.jpg and create a folder and add that image inside that folder
import shutil

for filename in os.listdir(test_dir):
    if filename.endswith('.jpg'): #checks that it is a jpg
        letter = filename.split('_')[0] # get the first part of the filename before '_'

        # create a folder for the letter if it doesn't exist
        letter_folder = os.path.join(test_dir, letter)
        if not os.path.exists(letter_folder):
            os.makedirs(letter_folder)
        shutil.move(os.path.join(test_dir, filename), os.path.join(letter_folder, filename)) #moves the image into the corresponding folder

print("Data re-organising complete")

Data re-organising complete


In [14]:
#imports
import os
import matplotlib.pyplot as plt
import torch
from sklearn.metrics import accuracy_score
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np

In [15]:
#model and train the CNN
'''
1. input data
2. preprocessing
  - standardize images(dont need to images from the dataset already comes 200x200)
  - color transformation
  - rotate image
3. feature extraction
4. ML model
'''

mean = [0.5,0.5,0.5]
std = [0.5,0.5,0.5]

#normalization
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std) #need to change the mean and std but im not sure change to what though
    #i think need to calc mean and std with like the train_loader len(dataset) thingy ?
])

train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [16]:
#train model

In [17]:
'''
1. Load and organize data into a usable format.
2. Resize and normalize the images to ensure consistency and optimal input for CNN.
3. Apply data augmentation on the training data to avoid overfitting and enhance generalization.
4. Split data into batches using DataLoader to handle larger datasets and speed up training.
'''

mean = torch.tensor([0.485, 0.456, 0.406])
std = torch.tensor([0.229, 0.224, 0.225])

#data augmentation transformation
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  #randomly flip the image horizontally
    transforms.RandomRotation(20),  #randomly rotate by a degree (-20 to 20)
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  #random color adjustments
    transforms.RandomResizedCrop(224),  #randomly crop and resize to 224x224
    transforms.ToTensor(),  #convert image to a tensor
    transforms.Normalize(mean=mean, std=std),  #normalize with mean and std
])

#transformation to resize, normalize, and convert images to a tensor
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),  #resize images to 224x224
    transforms.ToTensor(),  #convert the image to a pytorch tensor
    transforms.Normalize(mean=mean , std=std), #normalization
])

#load and transform dataset
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)

#create dataloader for batching the data
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [18]:
#possibly need to nn.sequential